# Tensorflow COTS

In [ ]:
# !pip install pandas
# !pip install tqdm
# !pip install ipywidgets
# !pip install jupyterlab_widgets 
# pip install ast
# pip install shutil
# pip install lxml
# # pip install xml

#####################
# !pip install joblib
# !pip install sklearn

In [ ]:
import warnings
warnings.filterwarnings("ignore")

sys.path.append('/kaggle/')

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import ast
import os
from shutil import copyfile
from lxml.etree import Element, SubElement, tostring
from xml.dom.minidom import parseString

###########################
# import json
# # from dicttoxml import dicttoxml
# import argparse

# # import torch
# import importlib
# import cv2 

# from sklearn.model_selection import GroupKFold
# from PIL import Image
# from string import Template
# from IPython.display import display

# import numpy as np
# import matplotlib.pyplot as plt
# from multiprocessing import Pool

# # from IPython.display import display, HTML
# # from matplotlib import animation, rc
# # rc('animation', html='jshtml')

# from itertools import groupby
# import pickle
# import glob

In [3]:
# check Torch and CUDA version
# !nvidia-smi
!nvcc --version
# print(f"Tensorflow: {tf.__version__}")

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [4]:
ROOT_DIR       = '/kaggle'                                     # directory root
DATA_DIR       = '/kaggle/input/TensorflowGreatBarrierReef'    # directory to save DATASET DATASET_PATH
WORK_DIR       = '/kaggle/working/' 

VOC_DIR        = '/kaggle/VOC2022'                             # directory to save images format p_voc
VOC_IMAGE_DIR  = '/kaggle/VOC2022/JPEGImages'                  # directory to save images
LABEL_TXT_DIR  = '/kaggle/VOC2022/ImageSets/Main'              # directory to save annotations
VOC_ANNO_DIR   = '/kaggle/VOC2022/annotations'                 # directory to save annotations

# VOC_LABEL_DIR      = '/kaggle/VOC2022/annotations/labels'      # directory to save labels
# VOC_XML_DIR        = '/kaggle/VOC2022/annotations/xmls'        # directory to save xmls

In [5]:
# Create dirs
!mkdir {VOC_DIR}
!mkdir -p {VOC_IMAGE_DIR}
!mkdir -p {LABEL_TXT_DIR}
!mkdir -p {VOC_ANNO_DIR}

# !mkdir {IMAGE_DIR}{DATASET_PATH}/train2017
# !mkdir {IMAGE_DIR}{DATASET_PATH}/val2017

mkdir: cannot create directory ‘/kaggle/VOC2022’: File exists


In [6]:
# FOLD      = 5 # which fold to train
# REMOVE_NOBBOX = True # remove images with no bbox

In [7]:
# # 
# df = pd.read_csv(f'{DATA_DIR}/train.csv')
# df.head(3)


In [8]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

In [9]:
def get_path(row):        
    im_str        = row['image_id']
    im_str_sum    = (len(im_str))
    im_name       = im_str[0] + '0'*(9-im_str_sum) + im_str[2:]
    im_path_name  = os.path.join(VOC_IMAGE_DIR, im_name)
    xml_path_name = os.path.join(VOC_ANNO_DIR,  im_name)
    
    row['image_path']   = f'{im_path_name}.jpg'   
    row['mxl_path']     = f'{xml_path_name}.xml'
    # row['txt_path']   = f'{xml_path_name}.txt'   
    
    row['im_name']      = f'{im_name}.jpg'
    # row['num_bbox']     = row['annotations'].count('x')    
    row['old_image_path'] = f'{DATA_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [10]:
def gen_xml(row):    
    obj_sum = row['num_bbox']
    
    if obj_sum > 0:
        
        path   = row['old_image_path']
        new_path   = row['image_path']
        copyfile(f'{path}', f'{new_path}')
 
        xmin_tuple = []
        ymin_tuple = []
        xmax_tuple = []
        ymax_tuple = []
        
        bbox_list  = row['bboxes']
        im_name    = row['im_name']
        xml_dst    = row['mxl_path']

        for i in range(0, obj_sum):
            x=bbox_list[i][0]
            y=bbox_list[i][1]
            w=bbox_list[i][2]
            h=bbox_list[i][3]

            xmin_tuple.append(x)
            ymin_tuple.append(y)
            xmax = x + w
            if xmax > 1280:
                xmax = 1280
            xmax_tuple.append(xmax)
            ymax = y + h
            if ymax > 720:
                ymax = 720
            ymax_tuple.append(ymax)        

        node_root = Element('annotation')
        node_folder = SubElement(node_root, 'folder')
        node_folder.text = 'VOC2022'

        node_filename = SubElement(node_root, 'filename')
        node_filename.text = im_name

        node_object_num = SubElement(node_root, 'object_num')
        node_object_num.text = str(obj_sum)

        node_size = SubElement(node_root, 'size')
        node_width = SubElement(node_size, 'width')
        node_width.text = "1280"

        node_height = SubElement(node_size, 'height')
        node_height.text = "720"

        for i in range(obj_sum):  
            node_object = SubElement(node_root, 'object')
            node_name = SubElement(node_object, 'name')
            node_name.text = "starfish"
            node_difficult = SubElement(node_object, 'difficult')
            node_difficult.text = '0'

            node_bndbox = SubElement(node_object, 'bndbox')
            node_xmin = SubElement(node_bndbox, 'xmin')
            node_xmin.text = str(xmin_tuple[i])
            node_ymin = SubElement(node_bndbox, 'ymin')
            node_ymin.text = str(ymin_tuple[i])
            node_xmax = SubElement(node_bndbox, 'xmax')
            node_xmax.text = str(xmax_tuple[i])
            node_ymax = SubElement(node_bndbox, 'ymax')
            node_ymax.text = str(ymax_tuple[i])

        xml = tostring(node_root)
        dom = parseString(xml)
        with open(xml_dst,'wb') as f:        
            f.write(dom.toprettyxml(indent='\t',encoding = 'utf-8'))
    else:
        pass

In [19]:
#
df = pd.read_csv(f'{DATA_DIR}/train.csv')

# Taken only annotated photos
df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))
df_train = df[df["num_bbox"]>0]

#Annotations 
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

# #Images resolution
# df_train["width"] = 1280
# df_train["height"] = 720

#Path of images
df_train = df_train.progress_apply(get_path, axis=1)
display(df_train.head(2))

  0%|          | 0/4919 [00:00<?, ?it/s]

  0%|          | 0/4919 [00:00<?, ?it/s]

  0%|          | 0/4919 [00:00<?, ?it/s]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,num_bbox,bboxes,image_path,mxl_path,im_name,old_image_path
16,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,"[[559, 213, 50, 32]]",/kaggle/VOC2022/JPEGImages/00000016.jpg,/kaggle/VOC2022/annotations/00000016.xml,00000016.jpg,/kaggle/input/TensorflowGreatBarrierReef/...
17,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,"[[558, 213, 50, 32]]",/kaggle/VOC2022/JPEGImages/00000017.jpg,/kaggle/VOC2022/annotations/00000017.xml,00000017.jpg,/kaggle/input/TensorflowGreatBarrierReef/...


In [23]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import ast
import os
from shutil import copyfile
from lxml.etree import Element, SubElement, tostring
from xml.dom.minidom import parseString

In [2]:
# 
import warnings
warnings.filterwarnings("ignore")

# import json
# # from dicttoxml import dicttoxml
# import argparse
# from lxml.etree import Element, SubElement, tostring
# from xml.dom.minidom import parseString
import pandas as pd
# # import torch
# import importlib
# import cv2 



# from sklearn.model_selection import GroupKFold
# from PIL import Image
# from string import Template
# from IPython.display import display

# import sys
# sys.path.append('/kaggle/')

# import numpy as np
# import matplotlib.pyplot as plt
# from multiprocessing import Pool

# # from IPython.display import display, HTML
# # from matplotlib import animation, rc
# # rc('animation', html='jshtml')

# from itertools import groupby
# import pickle
# import glob
# import shutil


In [ ]:
# data = (df.num_bbox>0).value_counts(normalize=True)*100
# print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [24]:
# 
df_train = df_train.progress_apply(gen_xml, axis=1)

  0%|          | 0/4919 [00:00<?, ?it/s]

In [ ]:
# print(df.keys())

# data = (df.num_bbox>0).value_counts(normalize=True)*100
# print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

# xmin_tuple = [d.get('x') for d in df['annotations'][9695]]
# xmin_tuple
# # type(xmin_tuple)
# # type(df['bboxes'][9695])
# list_anno = df['bboxes'][9695]
# len( df['bboxes'][9695])
# list_anno[0][0:]